In [32]:
from bs4 import BeautifulSoup
import cloudscraper
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import re

In [17]:
scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
html = scraper.get("https://www.hemnet.se/salda/bostader?item_types%5B%5D=villa&page=1&sold_age=12m").content
soup = BeautifulSoup(html, 'html.parser')

In [11]:
def get_house_links(soup):
    """Takes the soup of a page and returns a list of links to each house"""
    a_tags = soup.find_all('a',class_='sold-property-link js-sold-property-card-link')
    return [tag.get('href') for tag in a_tags]

def get_houses_info(links:list[str]):
    """Takes the link to a house and returns a dictionary with the house properties"""
    scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
    
    house_properties = []
    
    for link in links:
        html = scraper.get(link).content
        soup = BeautifulSoup(html, 'html.parser')
        
    


In [66]:
scraper_houses = cloudscraper.create_scraper()  # returns a CloudScraper instance
link = 'https://www.hemnet.se/salda/villa-6rum-alvsborg-goteborgs-kommun-pramgatan-7c-2628947802481352792'
html = scraper_houses.get(link).content
soup = BeautifulSoup(html, 'html.parser')




In [26]:
adress_unparsed = soup.find('h1',class_= 'hcl-heading hcl-heading--size1')
tag_to_remove = adress_unparsed.find('span',class_='sold-property__address-prefix')
tag_to_remove.decompose()
text = adress_unparsed.text.split('\n')
address = ' '.join([x for x in text if x])



'  Pråmgatan 7C'

In [67]:
adress_header_unparsed = soup.find('h1',class_= 'hcl-heading hcl-heading--size1')
adress_header_text = adress_unparsed.get_text()
adress_text_prefix = re.sub(r'\s+', ' ', adress_header_text)
adress_text_prefix = adress_text_prefix.strip()

adress_metadata_unparsed = soup.find('p',class_= 'sold-property__metadata qa-sold-property-metadata')
adress_metadata_unparsed.find('span',class_= 'property-icon property__description-icon').decompose()
adress_metadata_unparsed.find('time').decompose()
adress_metadata_text = adress_metadata_unparsed.get_text()
adress_metadata_text = re.sub(r'\s+', ' ', adress_metadata_text)
adress_metadata_text = re.search(r' - (.*?) - ', adress_metadata_text)


adress_text = f'{adress_text_prefix}, {adress_metadata_text.group(1)}' # gets the whole adress for a house




Pråmgatan 7C, Älvsborg, Göteborgs kommun


In [21]:


def get_coordinates(addresses:list[str]):
    geolocator = Nominatim(user_agent="House_price_prediction")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    locations = []
    
    for address in addresses:
        location = geolocator.geocode(address)
        locations.append((location.latitude, location.longitude))



(57.7079695, 12.8036417)
